# PFI after tuning on ABIDE versus original

In [1]:
sABIDE2 = '/archive/bioinformatics/DLLab/CooperMellema/results/Autism/ABIDE_Tuned/ABIDE2/PFI_Tuned_64Permutations/*.csv'
sABIDE1 = '/archive/bioinformatics/DLLab/CooperMellema/results/Autism/ABIDE_Tuned/ABIDE1/PFI_Tuned_64Permutations/*.csv'
s64 = '/project/bioinformatics/DLLab/Cooper/Code/AutismProject/JournalPaperData/AtlasResolutionComparison8Permutations/64ROI_MeanImportances.p'
s122 = '/project/bioinformatics/DLLab/Cooper/Code/AutismProject/JournalPaperData/AtlasResolutionComparison8Permutations/122ROI_MeanImportances.p'
s197 = '/project/bioinformatics/DLLab/Cooper/Code/AutismProject/JournalPaperData/AtlasResolutionComparison8Permutations/197ROI_MeanImportances.p'

In [2]:
import pandas as pd
import glob
import os
import pickle as pkl
import numpy as np

In [156]:
lsABIDE1 = glob.glob(sABIDE1)
lsABIDE2 = glob.glob(sABIDE2)
pd64 = pd.DataFrame.from_dict(pkl.load(open(s64, 'rb')), orient='index').sort_values(by=0, ascending=False)
pd122 = pd.DataFrame.from_dict(pkl.load(open(s64, 'rb')), orient='index').sort_values(by=0, ascending=False)
pd197 = pd.DataFrame.from_dict(pkl.load(open(s64, 'rb')), orient='index').sort_values(by=0, ascending=False)

In [157]:
pdABIDE1_64=pd.concat([pd.read_csv(x, index_col=0).iloc[:,0:1] for x in lsABIDE1 if int(x.split('Model')[-1].split('_')[0]) in [i*3+1 for i in range(5)]], axis=1).mean(axis=1).to_frame().sort_values(by=0, ascending=False)
pdABIDE1_122=pd.concat([pd.read_csv(x, index_col=0).iloc[:,0:1] for x in lsABIDE1 if int(x.split('Model')[-1].split('_')[0]) in [i*3+2 for i in range(5)]], axis=1).mean(axis=1).to_frame().sort_values(by=0, ascending=False)
pdABIDE1_197=pd.concat([pd.read_csv(x, index_col=0).iloc[:,0:1] for x in lsABIDE1 if int(x.split('Model')[-1].split('_')[0]) in [i*3+3 for i in range(5)]], axis=1).mean(axis=1).to_frame().sort_values(by=0, ascending=False)
pdABIDE2_64=pd.concat([pd.read_csv(x, index_col=0).iloc[:,0:1] for x in lsABIDE2 if int(x.split('Model')[-1].split('_')[0]) in [i*3+1 for i in range(5)]], axis=1).mean(axis=1).to_frame().sort_values(by=0, ascending=False)
pdABIDE2_122=pd.concat([pd.read_csv(x, index_col=0).iloc[:,0:1] for x in lsABIDE2 if int(x.split('Model')[-1].split('_')[0]) in [i*3+2 for i in range(5)]], axis=1).mean(axis=1).to_frame().sort_values(by=0, ascending=False)
pdABIDE2_197=pd.concat([pd.read_csv(x, index_col=0).iloc[:,0:1] for x in lsABIDE2 if int(x.split('Model')[-1].split('_')[0]) in [i*3+3 for i in range(5)]], axis=1).mean(axis=1).to_frame().sort_values(by=0, ascending=False)

In [158]:
for a1, a2 in [(pd64, pdABIDE1_64), (pd122, pdABIDE1_122), (pd197, pdABIDE1_197), (pd64, pdABIDE2_64), (pd122, pdABIDE2_122), (pd197, pdABIDE2_197)]:
    s1=set(a1.loc[(a1>3*a1[0].std()).values.flatten()].index)
    s2=set(a2.loc[(a2>3*a2[0].std()).values.flatten()].index)
    print(100*len(s1.intersection(s2))/min(len(s1), len(s2)))

30.232558139534884
2.5641025641025643
0.0
33.333333333333336
4.081632653061225
87.75510204081633


In [171]:
from scipy.stats import spearmanr, pearsonr


In [212]:
lsSP=[]
lsPR=[]
for a1, a2 in [(pd64, pdABIDE1_64), (pd122, pdABIDE1_122), (pd197, pdABIDE1_197), (pd64, pdABIDE2_64), (pd122, pdABIDE2_122), (pd197, pdABIDE2_197)]:
    try:
        a2=a2.sort_index().loc[a1.index]
        a1=a1.sort_index()
        thresh = 3*np.std(np.concatenate([a1.values, a2.values]))
        sp=spearmanr(abs(a1.loc[np.bitwise_or((abs(a1)>thresh).values, (abs(a2)>thresh).values).flatten()]), abs(a2.loc[np.bitwise_or((abs(a1)>thresh).values, (abs(a2)>thresh).values).flatten()]))
        pr=pearsonr(abs(a1.loc[np.bitwise_or((abs(a1)>thresh).values, (abs(a2)>thresh).values).flatten()].values.flatten()), abs(a2.loc[np.bitwise_or((abs(a1)>thresh).values, (abs(a2)>thresh).values).flatten()].values.flatten()))
        print(sp)
        print(pr)
        lsSP.append(abs(sp[0]))
        lsPR.append(abs(pr[0]))
    except:
        print('ERROR')
print('NOTE: Error in ABIDE 1, 197 ROIs (third entry), PFI incomplete due to memory overflow, ignored going forward')

SpearmanrResult(correlation=-0.03932089712111359, pvalue=0.744750683137488)
(-0.27904375107956475, 0.018445749810039356)
SpearmanrResult(correlation=-0.6575391154502028, pvalue=1.9639792233137573e-10)
(-0.6178145917337516, 4.520247675059026e-09)
SpearmanrResult(correlation=-0.7414979757085021, pvalue=6.664212475491701e-08)
(-0.6585298586789411, 5.184391822540278e-06)
SpearmanrResult(correlation=-0.06759168182357869, pvalue=0.6141669770932461)
(-0.19345352668860372, 0.14566621351428194)
SpearmanrResult(correlation=-0.6649766715505305, pvalue=1.5271981798926797e-09)
(-0.5655334305258578, 9.169555132414826e-07)
SpearmanrResult(correlation=-0.6270926797242586, pvalue=2.5204842990860495e-05)
(-0.7096173094803144, 6.091914935574851e-07)
NOTE: Error in ABIDE 1, 197 ROIs (third entry), PFI incomplete due to memory overflow


In [216]:
np.mean(lsSP)

-0.4663365035630311

In [215]:
np.mean(lsPR)

-0.5039987446978389